In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox
import pyrosm as py
import ast

import haversine as hs
import requests
import json
import googlemaps
from datetime import datetime

from API import K

In [4]:
df_pr['P+R']=True
df_pr.Day_price.replace('','free',inplace=True)

In [35]:
def find_nearest_parking(origin, destination, parking_time, display_dist=False, disabled=False, woman=False, family=False, expected_arrival=datetime.now()):
    """Function to serve as a decision making tool to find the nearest parkingplace in Munich given a address,
    and a dataframe with parkings that contains for each entry at least a lon and lat column 
    and optionally an expected arrival time (by default set to datetime.now()"""
    ############################################################################################
    ##step1: reading in data and preparing data    
    ############################################################################################
    #readin dataframe
    df=pd.read_excel('Data/P_R_Datenbank_2019_ohne.xlsx')
    #create needed lat and lon if normal address is input
    r= requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={destination}&key={K}")
    #unpack
    results = json.loads(r.content)
    #save lat, lon (to calculate bird_dist) and district (to calculate street parking tariffs)
    district = results['results'][0]['address_components'][2]['long_name']
    lat_r = results['results'][0]['geometry']['location']['lat']
    lon_r = results['results'][0]['geometry']['location']['lng']
    
    parking_time=tuple(map(int, parking_time.split(', ')))

    #create new columns to fill later
    df['bird_dist'] = 0
    df['Driving to parking'] = 0
    
    df['est_time_walking'] = 0
    df['est_time_bicycling'] = 0
    df['est_time_transit'] = 0
    
    if display_dist:
        df['est_dist_walking'] = 0
        df['est_dist_bicycling'] = 0
        df['est_dist_transit'] = 0
    
    #combine lat and lon in one column
    df['lat_lon']=list(zip(df.lat, df.lon))
    
    ############################################################################################
    #step 2: calculate distance as the crow flies from dest to all parkings
    ############################################################################################
    
    df['bird_dist'] = df.lat_lon.apply(lambda p: hs.haversine((lat_r,lon_r),p))
    
    #sort values based on bird_dist, reset index
    df.sort_values(by='bird_dist',inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    ############################################################################################
    #step 3:calculate travel time and approx costs if they where to drive directly to dest
    ############################################################################################

    r = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=driving&key={K}")
    #unpack
    results = json.loads(r.content)
    legs = results.get("routes").pop(0).get("legs")
    df['Driving to destination'] = legs[0].get("duration")['text']
    
    costsin1 = ['Milbershofen', 'Trudinger-Riem', 'Schwanthalerhöhe', 'Ludwigsvorstadt', 'Isarvorstadt', 'Au', 'Haidhausen',
                'Au-haidhausen', 'Giesing', 'Steinhausen', 'Tivoli', 'Maxvorstadt', "Schwabing-west", "Glockenbach", "Untersendling"]
    costsin2 = ['Moosach', 'Neuhausen', "Sendling-Westpark", "Giesling"]
    
    if ('Altstadt' in district) or ('Lehel' in district):
        df['Current costs streetparking'] = round((parking_time[0]*2.5+(parking_time[1]/60)*2.5),2)
    elif district in costsin1:
        df['Current costs streetparking'] = round(max((parking_time[0]*1+(parking_time[1]/60)*1), 6),2)
        df['Planned costs streetparking'] = round(max((parking_time[0]*1.9+(parking_time[1]/60)*1.9), 12),2)
    elif district in costsin2:
        df['Planned costs streetparking'] = round(max((parking_time[0]*1.9+(parking_time[1]/60)*1.9), 12),2)
    else:
        df['Current costs streetparking'] = 'free'
        
    ############################################################################################
    #step 4:calculate travel times to final dest for 5 nearest parkings for 3 forms of transport
    ############################################################################################
    
    for i in range (5):
        r = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={df.lat[i]},{df.lon[i]}&mode=driving&key={K}")
            #unpack
        results = json.loads(r.content)
        legs = results.get("routes").pop(0).get("legs")
        df['Driving to parking'][i] = legs[0].get("duration")['text']
            
        for j in ['walking','bicycling','transit']:
            #run google api and unpack relevant variables
            r = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?origin={df.lat[i]},{df.lon[i]}&destination={destination}&mode={j}&key={K}")
            #unpack
            results = json.loads(r.content)
            legs = results.get("routes").pop(0).get("legs")
            dur_dist=(legs[0].get("duration"), legs[0].get("distance"))
            #save data in previously assigned columns
            df['est_time_'+str(j)][i] = dur_dist[0]['text']
            if display_dist:
                df['est_dist_'+str(j)][i] = dur_dist[1]['text']
    
    ############################################################################################
    #step 5: final output data selection and preparation
    ############################################################################################
    
    #only first 5 are interesting, drop irrelevant columns
    df=df[:5].drop(columns=['lat','lon','Bahnhof','BahnhofID','GlobaleID', 'Name','Niveau,N,10,0','bird_dist','MVTT_x','MVTT_y', "Georeferenz", "Name DIVA", "lat_lon",'Entrance'])
    
    df.rename(columns={"Alternative_name": "Name"}, inplace=True)
    
    #subset availability
    availability = df.filter(regex='OCC_')
    availability = availability.join(df.loc[:,'Name'], lsuffix='_caller', rsuffix='_other')
    availability.set_index('Name', drop=True, inplace=True)
    
    to_k =['Name', 'Driving to parking','est_time_walking','est_time_bicycling', 'est_time_transit', 'Day_price',
           'Ticket_for_10', 'Month_ticket', 'Year_ticket','Driving to destination']
    
    if 'Current costs streetparking' in df.columns:
        to_k.append('Current costs streetparking')
    if 'Planned costs streetparking' in df.columns:
        to_k.append('Planned costs streetparking')
        
    if woman:
        to_k.append('P_women')
    if disabled:
        to_k.append('P_invalid')
    if family:
        to_k.append('P_family')
        
    to_k.append('Capacity')
    to_k.append('Link')
    df = df[to_k] #Create new dataframe with columns in the order you want
    
    #remove from other dataframe
    df.drop(list(df.filter(regex = 'OCC_')), axis = 1, inplace = True)
    return(df, availability)


## Create a random point
Random point somewhere in Munich, that might be a destination, to use to test function

In [5]:
#random point somewhere in Munich to tryout script
import random

latlon_r=(random.uniform(48.248116, 48.061624),random.uniform(11.722910, 11.360777))
print("Random point was selected", (latlon_r))

Random point was selected (48.124490882797616, 11.37148850633771)


In [13]:
def find_nearest_parking(origin, destination, parking_time, display_dist=False, disabled=False, woman=False, family=False, expected_arrival=datetime.now()):
    """Function to serve as a decision making tool to find the nearest parkingplace in Munich given a address,
    and a dataframe with parkings that contains for each entry at least a lon and lat column 
    and optionally an expected arrival time (by default set to datetime.now()"""
    ############################################################################################
    ##step1: reading in data and preparing data    
    ############################################################################################
    #readin dataframe
    df=pd.read_excel('Data/P_R_Datenbank_2019_ohne.xlsx')
    #create needed lat and lon if normal address is input
    r= requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={destination}&key={K}")
    #unpack
    results = json.loads(r.content)
    #save lat, lon (to calculate bird_dist) and district (to calculate street parking tariffs)
    district = results['results'][0]['address_components'][2]['long_name']
    lat_r = results['results'][0]['geometry']['location']['lat']
    lon_r = results['results'][0]['geometry']['location']['lng']
    
    parking_time=tuple(map(int, parking_time.split(', ')))
    #create new columns to fill later
    df['bird_dist'] = 0
    df['Driving to parking'] = 0
    
    df['est_time_walking'] = 0
    df['est_time_bicycling'] = 0
    df['est_time_transit'] = 0
    
    if display_dist:
        df['est_dist_walking'] = 0
        df['est_dist_bicycling'] = 0
        df['est_dist_transit'] = 0
    
    #combine lat and lon in one column
    df['lat_lon']=list(zip(df.lat, df.lon))
    
    ############################################################################################
    #step 2: calculate distance as the crow flies from dest to all parkings
    ############################################################################################
    
    df['bird_dist'] = df.lat_lon.apply(lambda p: hs.haversine((lat_r,lon_r),p))
    
    #sort values based on bird_dist, reset index
    df.sort_values(by='bird_dist',inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    ############################################################################################
    #step 3:calculate travel time and approx costs if they where to drive directly to dest
    ############################################################################################

    r = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=driving&key={K}")
    #unpack
    results = json.loads(r.content)
    legs = results.get("routes").pop(0).get("legs")
    df['Driving to destination'] = legs[0].get("duration")['text']
    
    costsin1 = ['Milbershofen', 'Trudinger-Riem', 'Schwanthalerhöhe', 'Ludwigsvorstadt', 'Isarvorstadt', 'Au', 'Haidhausen',
                'Au-haidhausen', 'Giesing', 'Steinhausen', 'Tivoli', 'Maxvorstadt', "Schwabing-west", "Glockenbach", "Untersendling"]
    costsin2 = ['Moosach', 'Neuhausen', "Sendling-Westpark", "Giesling"]
    
    if ('Altstadt' in district) or ('Lehel' in district):
        df['Current costs streetparking'] = round((parking_time[0]*2.5+(parking_time[1]/60)*2.5),2)
    elif district in costsin1:
        df['Current costs streetparking'] = round(max((parking_time[0]*1+(parking_time[1]/60)*1), 6),2)
        df['Planned costs streetparking'] = round(max((parking_time[0]*1.9+(parking_time[1]/60)*1.9), 12),2)
    elif district in costsin2:
        df['Planned costs streetparking'] = round(max((parking_time[0]*1.9+(parking_time[1]/60)*1.9), 12),2)
    else:
        df['Current costs streetparking'] = 'free'
        
    ############################################################################################
    #step 4:calculate travel times to final dest for 5 nearest parkings for 3 forms of transport
    ############################################################################################
    
    for i in range (5):
        r = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={df.lat[i]},{df.lon[i]}&mode=driving&key={K}")
            #unpack
        results = json.loads(r.content)
        legs = results.get("routes").pop(0).get("legs")
        df['Driving to parking'][i] = legs[0].get("duration")['text']
            
        for j in ['walking','bicycling','transit']:
            #run google api and unpack relevant variables
            r = requests.get(f"https://maps.googleapis.com/maps/api/directions/json?origin={df.lat[i]},{df.lon[i]}&destination={destination}&mode={j}&key={K}")
            #unpack
            results = json.loads(r.content)
            legs = results.get("routes").pop(0).get("legs")
            dur_dist=(legs[0].get("duration"), legs[0].get("distance"))
            #save data in previously assigned columns
            df['est_time_'+str(j)][i] = dur_dist[0]['text']
            if display_dist:
                df['est_dist_'+str(j)][i] = dur_dist[1]['text']
    
    ############################################################################################
    #step 5: final output data selection and preparation
    ############################################################################################
    
    #only first 5 are interesting, drop irrelevant columns
    df=df[:5].drop(columns=['lat','lon','Bahnhof','BahnhofID','GlobaleID', 'Name','Niveau,N,10,0','bird_dist','MVTT_x','MVTT_y', "Georeferenz", "Name DIVA", "lat_lon",'Entrance'])
    
    df.rename(columns={"Alternative_name": "Name"}, inplace=True)
    
        #subset availability
    availability = df.filter(regex='OCC_')
    availability = availability.join(df.loc[:,'Name'], lsuffix='_caller', rsuffix='_other')
    availability.set_index('Name', drop=True, inplace=True)
    
    to_k =['Name', 'Driving to parking','est_time_walking','est_time_bicycling', 'est_time_transit', 'Day_price',
           'Ticket_for_10', 'Month_ticket', 'Year_ticket','Driving to destination']
    
    if 'Current costs streetparking' in df.columns:
        to_k.append('Current costs streetparking')
    if 'Planned costs streetparking' in df.columns:
        to_k.append('Planned costs streetparking')
        
    if woman:
        to_k.append('P_women')
    if disabled:
        to_k.append('P_invalid')
    if family:
        to_k.append('P_family')
        
    to_k.append('Capacity')
    to_k.append('Link')
    df = df[to_k] #Create new dataframe with columns in the order you want
    
    #remove from other dataframe
    df.drop(list(df.filter(regex = 'OCC_')), axis = 1, inplace = True)
    return(df, availability)

In [36]:
results=find_nearest_parking('Schaufeleinstraße 46, Munich','Arcisstrasse 23, Munich', "7, 10")

C:\Users\Fabia\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Fabia\anaconda3\envs\gds\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\Fabia\anaconda3\envs\gds\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [37]:
df1=results[0]
df2=results[1]

In [38]:
df1

,Name,Driving to parking,est_time_walking,est_time_bicycling,est_time_transit,Day_price,Ticket_for_10,Month_ticket,Year_ticket,Driving to destination,Current costs streetparking,Planned costs streetparking,Capacity,Link
0,P+R Heimeranplatz,6 mins,55 mins,18 mins,25 mins,1.5,14.5,19,190,16 mins,7.17,13.62,310,https://www.mvv-muenchen.de/plaene-bahnhoefe/b...
1,P+R Olympiazentrum,14 mins,53 mins,16 mins,20 mins,1.5,14.5,19,190,16 mins,7.17,13.62,274,https://www.mvv-muenchen.de/plaene-bahnhoefe/b...
2,P+R Westfriedhof,14 mins,56 mins,18 mins,25 mins,1.5,14.5,19,190,16 mins,7.17,13.62,116,https://www.mvv-muenchen.de/plaene-bahnhoefe/b...
3,P+R Westfriedhof TG,14 mins,55 mins,16 mins,24 mins,1.5,14.5,19,190,16 mins,7.17,13.62,221,https://www.mvv-muenchen.de/plaene-bahnhoefe/b...
4,P+R Oberwiesenfeld,12 mins,1 hour 7 mins,20 mins,23 mins,1.0,9.5,11,110,16 mins,7.17,13.62,146,https://www.mvv-muenchen.de/plaene-bahnhoefe/b...


In [39]:
df2

,OCC_h05,OCC_h06,OCC_h07,OCC_h08,OCC_h09,OCC_h10,OCC_h11,OCC_h12,OCC_h13,OCC_h14,OCC_h15,OCC_h16,OCC_h17,OCC_h18,OCC_h19,OCC_h20,OCC_h21,OCC_h22
Name,,,,,,,,,,,,,,,,,,
P+R Heimeranplatz,5.0,20.0,40.0,60.0,80.0,95.0,95.0,95.0,95.0,85.0,80.0,70.0,50.0,30.0,20.0,10.0,10.0,10.0
P+R Olympiazentrum,5.0,40.0,60.0,75.0,85.0,95.0,100.0,100.0,100.0,90.0,90.0,75.0,70.0,40.0,20.0,10.0,10.0,10.0
P+R Westfriedhof,5.0,40.0,80.0,95.0,100.0,100.0,100.0,100.0,95.0,90.0,85.0,75.0,60.0,50.0,20.0,10.0,10.0,10.0
P+R Westfriedhof TG,5.0,20.0,40.0,60.0,75.0,80.0,85.0,85.0,85.0,85.0,75.0,70.0,60.0,50.0,20.0,10.0,10.0,10.0
P+R Oberwiesenfeld,5.0,20.0,70.0,70.0,85.0,95.0,95.0,85.0,85.0,85.0,80.0,70.0,60.0,30.0,20.0,10.0,10.0,10.0


In [ ]:
#Want to plot the occupancy
a=results[1]

#Delete parkings of which we don't have the occupancy
a.dropna(how="all", axis=1, inplace=True)

#Give columns plottable name
a.columns=list(np.arange(5,23,1))

#setup grid
fig,ax=plt.subplots(1,1,dpi=100, figsize=(8,5))
tcks=[]
locs=[]
for i in range(5,23,3):
    tcks.append(str(i)+"h")
    locs.append(i)
plt.xticks(locs, tcks)

#plot vertical line at current time if 
if 5<datetime.now().hour<22:
    plt.axvline(x=(datetime.now().hour+(datetime.now().minute/60)), linestyle="--", alpha=0.5, color='r', label='Current_time')
ax.set_xlabel('Time of day')
ax.set_ylabel('Expected occupancy [%]')
ax.set_title('Expected occupancy during the day')

for i,j in enumerate(a.index):
    print()
    ax=a.iloc[i].plot(alpha=0.5)

# legend = plt.legend(loc="lower left", edgecolor="black", fontsize=8, framealpha=0)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10),
          fancybox=True, shadow=True, ncol=5)

In [21]:
# as a function
def vis_occ(df1):
    a=results[1]

    #Delete parkings of which we don't have the occupancy
    a.dropna(how="all", axis=1, inplace=True)

    #Give columns plottable name
    a.columns=list(np.arange(5,23,1))

    #setup grid
    fig,ax=plt.subplots(1,1,dpi=100, figsize=(8,5))
    tcks=[]
    locs=[]
    for i in range(5,23,3):
        tcks.append(str(i)+"h")
        locs.append(i)
    plt.xticks(locs, tcks)

    #plot vertical line at current time if 
    if 5<datetime.now().hour<22:
        plt.axvline(x=(datetime.now().hour+(datetime.now().minute/60)), linestyle="--", alpha=0.5, color='r', label='Current_time')
    ax.set_xlabel('Time of day')
    ax.set_ylabel('Expected occupancy [%]')
    ax.set_title('Expected occupancy during the day')

    for i,j in enumerate(a.index):
        print()
        ax=a.iloc[i].plot(alpha=0.5)

    # legend = plt.legend(loc="lower left", edgecolor="black", fontsize=8, framealpha=0)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10),
          fancybox=True, shadow=True, ncol=5)
    return(fig)

In [63]:
r= requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={'arcisstrasse 23,Munich,Germany'}&key={K}")
results = json.loads(r.content)
district = results['results'][0]['address_components'][2]['long_name']
print(district)

Maxvorstadt


In [28]:
a="1, 2"
parking_time=tuple(map(int, a.split(', ')))
parking_time

(1, 2)

## Lookup other potential data suppliers

In [38]:
df_11=pd.read_excel('Data/Parkings_in_and_near.xlsx')

In [ ]:
for i in df_11.columns:
    print(i, df_11[i].count(), df_11[i].nunique())

In [ ]:
df_11.operator.unique()

In [ ]:
df_12=df_11.copy()
df_12.dropna(subset=['operator'],inplace=True)
df_12.drop(columns=['tags'],inplace=True)
df_12[df_12['operator'].str.contains("Contipark")].dropna(how='all',axis=1).operator

In [ ]:
df_12[df_12['operator'].str.contains("Contipark")].dropna(how='all',axis=1)

In [54]:
a=0
for i in df_11.capacity:
    if type(i) != str:
        a= a + float(i)

In [55]:
a

nan

In [76]:
a='Altstadt'
if ('Altstadt' in a) or ('Lehel' in a):
    print(True)

True
